## **Лабораторная работа №8 (Проведение исследований моделями обнаружения и распознавания объектов)**

In [2]:
import os
import cv2
import numpy as np
from glob import glob
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

c:\Users\missm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**2.	Создание бейзлайна и оценка качества**

**a.	Обучить модели (как сверточные так и «трансформерные») из torchvision для выбранного набора данных**


In [1]:
from ultralytics import YOLO

# a. Модель YOLOv8n
model_base = YOLO('yolov8n.pt')
model_base.train(data='dataset.yaml', epochs=3, imgsz=640, project='lab8', name='yolo_base')


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\missm\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kob

train: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\train... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<00:00, 300.47it/s]

train: New cache created: C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\train.cache



c:\Users\missm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.20.1 ms, read: 2.00.6 MB/s, size: 20.9 KB)


...

                   all         73       1038      0.884      0.622      0.718      0.507



3 epochs completed in 0.120 hours.
Optimizer stripped from lab8\yolo_base4\weights\last.pt, 6.2MB
Optimizer stripped from lab8\yolo_base4\weights\best.pt, 6.2MB

Validating lab8\yolo_base4\weights\best.pt...
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.81s/it]


                   all         73       1038      0.884      0.622      0.717      0.505
                   WBC         72         74      0.979          1      0.994      0.802
                   RBC         70        864      0.673      0.865      0.791      0.513
             Platelets         49        100          1          0      0.366        0.2
Speed: 1.0ms preprocess, 91.7ms inference, 0.0ms loss, 12.0ms postprocess per image
Results saved to lab8\yolo_base4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200CB6F5A60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

**b.	Оценить качество моделей по выбранным метрикам на выбранном наборе данных**



In [2]:
metrics_base = model_base.val()
print(metrics_base)

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 144.121.3 MB/s, size: 20.6 KB)


val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.64s/it]


                   all         73       1038      0.884      0.622      0.717      0.505
                   WBC         72         74      0.979          1      0.994      0.802
                   RBC         70        864      0.673      0.865      0.791      0.513
             Platelets         49        100          1          0      0.366        0.2
Speed: 1.1ms preprocess, 90.8ms inference, 0.0ms loss, 11.1ms postprocess per image
Results saved to lab8\yolo_base42
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200E1135A90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012

| Class      | Precision | Recall | F1-score | mAP50 |
|------------|-----------|--------|----------|--------|
| **WBC**        | 0.979     | 1.000  | 0.994    | 0.802  |
| **RBC**        | 0.673     | 0.865  | 0.791    | 0.513  |
| **Platelets**  | 1.000     | 0.000  | 0.366    | 0.200  |


WBC (белые клетки крови): модель определяет очень хорошо.
RBC (красные клетки): немного ниже precision, но recall хороший.
Platelets (тромбоциты): precision максимальный (1.0), но recall — 0, что говорит о переобученииили отсутствии успешных предсказаний для этого класса. Возможно, нужно больше данных этого типа или сбалансировать классы.

**3.	Улучшение бейзлайна**

**a.	Сформулировать гипотезы (аугментации данных, подбор моделей, подбор гиперпараметров и т.д.)**


- Использовать более крупную модель (YOLOv8s вместо YOLOv8n)
- Увеличить разрешение изображений
- Увеличить количество эпох
- Применить data augmentation


**b.	Проверить гипотезы**

**c.	Сформировать улучшенный бейзлайн по результатам проверки гипотез**

В ходе эксперимента и проверки гипотез были сформулирована следующий улучшенный бизлайн

**d.	Обучить модели с улучшенным бейзлайном на выбранном наборе данных**


In [5]:
model_imp = YOLO('yolov8s.pt')
model_imp.train(data='dataset.yaml', epochs=5, imgsz=768, project='lab8', name='yolov8s_imp', augment=True)

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_imp2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tru

train: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\train.cache... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.30.1 ms, read: 44.34.6 MB/s, size: 20.9 KB)



val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]

Plotting labels to lab8\yolov8s_imp2\labels.jpg... 


...

                   all         73       1038      0.862      0.881      0.905      0.611



5 epochs completed in 0.524 hours.
Optimizer stripped from lab8\yolov8s_imp2\weights\last.pt, 22.5MB
Optimizer stripped from lab8\yolov8s_imp2\weights\best.pt, 22.5MB

Validating lab8\yolov8s_imp2\weights\best.pt...
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:18<00:00, 26.05s/it]


                   all         73       1038      0.856      0.893      0.914      0.622
                   WBC         72         74      0.968          1      0.994      0.805
                   RBC         70        864      0.727      0.845      0.853      0.593
             Platelets         49        100      0.874      0.834      0.895      0.468
Speed: 4.1ms preprocess, 1045.2ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to lab8\yolov8s_imp2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200BF02D550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

**e.	Оценить качество моделей с улучшенным бейзлайном по выбранным метрикам на выбранном наборе данных**

In [6]:
metrics_imp = model_imp.val()
print(metrics_imp)

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 76.052.7 MB/s, size: 20.4 KB)


val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [01:17<00:00, 15.45s/it]


                   all         73       1038      0.856      0.893      0.914      0.622
                   WBC         72         74      0.968          1      0.994      0.805
                   RBC         70        864      0.727      0.845      0.853      0.593
             Platelets         49        100      0.874      0.834      0.895      0.468
Speed: 2.8ms preprocess, 1036.6ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to lab8\yolov8s_imp22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200BEE8BE60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012



| Класс         | Precision | Recall | F1-score | mAP50 |
| ------------- | --------- | ------ | -------- | ----- |
| **WBC**       | 0.968     | 1.000  | 0.984    | 0.994 |
| **RBC**       | 0.727     | 0.845  | 0.781    | 0.853 |
| **Platelets** | 0.874     | 0.834  | 0.854    | 0.895 |


Класс WBC (лейкоциты) демонстрирует наивысшую точность и полноту — Precision = 0.968, Recall = 1.000, F1-score = 0.984, mAP50 = 0.994. Это свидетельствует о почти безошибочном распознавании этого класса моделью: почти все объекты WBC обнаруживаются и классифицируются корректно.

Класс RBC (эритроциты) показывает наихудшие результаты по всем метрикам: Precision = 0.727, Recall = 0.845, F1-score = 0.781, mAP50 = 0.853. Это указывает на относительно высокое количество ложноположительных результатов (низкая precision) и некоторые пропущенные объекты (неидеальный recall), что говорит о необходимости улучшения распознавания этого класса.

Класс Platelets (тромбоциты) имеет сбалансированные показатели: Precision = 0.874, Recall = 0.834, F1-score = 0.854, mAP50 = 0.895. Модель уверенно распознаёт тромбоциты, хотя и с меньшей точностью, чем лейкоциты.



**f.	Сравнить результаты моделей с улучшенным бейзлайном в сравнении с результатами из пункта 2**

Сравнение базовой модели и улучшенного бейзлайна  показывает, что несмотря на отдельные улучшения, общая эффективность новой версии модели вызывает сомнения. В базовой модели наблюдаются высокие показатели по всем метрикам для класса WBC: Precision = 0.968, Recall = 1.000, F1-score = 0.984 и mAP50 = 0.994, что указывает на стабильную и точную работу. RBC определяются с умеренной точностью, но достаточно уверенно (F1 = 0.781), а Platelets распознаются сбалансированно (F1 = 0.854). В улучшенном бейзлайне для класса WBC Precision даже выше — 0.979, а Recall остаётся идеальным, но качество локализации ухудшается (mAP50 = 0.802). Для RBC Recall улучшается (0.865), но Precision падает, и mAP50 сильно снижается (до 0.513), что говорит о неточной локализации. Особенно критична ситуация с Platelets — модель показывает Precision = 1.000, но Recall = 0.000, то есть она полностью игнорирует данный класс, несмотря на отсутствие ложных срабатываний. Это отражается в крайне низких F1-score (0.366) и mAP50 (0.200).

**g.	Сделать выводы**

Несмотря на некоторые локальные улучшения (в частности, повышение точности WBC и Recall RBC), улучшенный бейзлайн показывает деградацию общего качества, особенно в выявлении и локализации тромбоцитов. В практическом применении базовая модель остаётся более надёжной и сбалансированной, обеспечивая уверенное определение всех классов.

**4.	Имплементация алгоритма машинного обучения**

**a.	Самостоятельно имплементировать модели машинного обучения**

**b.	Обучить имплементированные модели на выбранном наборе данных**



In [7]:
model_custom = YOLO('yolov8n.pt')
model_custom.train(data='dataset.yaml', epochs=5, imgsz=640, freeze=10, project='lab8', name='custom_frozen')
metrics_custom = model_custom.val()
print(metrics_custom)

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_frozen, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tru

train: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\train.cache... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 73.247.9 MB/s, size: 20.9 KB)



val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]

Plotting labels to lab8\custom_frozen\labels.jpg... 


...


5 epochs completed in 0.177 hours.
Optimizer stripped from lab8\custom_frozen\weights\last.pt, 6.2MB
Optimizer stripped from lab8\custom_frozen\weights\best.pt, 6.2MB

Validating lab8\custom_frozen\weights\best.pt...
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:12<00:00,  4.04s/it]


                   all         73       1038      0.765       0.78      0.845       0.55
                   WBC         72         74      0.804      0.986      0.981      0.767
                   RBC         70        864      0.624      0.914      0.817      0.541
             Platelets         49        100      0.868       0.44      0.738      0.342
Speed: 1.9ms preprocess, 135.0ms inference, 0.0ms loss, 15.4ms postprocess per image
Results saved to lab8\custom_frozen
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.2 ms, read: 70.034.2 MB/s, size: 20.4 KB)


val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:10<00:00,  2.13s/it]


                   all         73       1038      0.765       0.78      0.845       0.55
                   WBC         72         74      0.804      0.986      0.981      0.767
                   RBC         70        864      0.624      0.914      0.817      0.541
             Platelets         49        100      0.868       0.44      0.738      0.342
Speed: 1.4ms preprocess, 122.6ms inference, 0.0ms loss, 11.4ms postprocess per image
Results saved to lab8\custom_frozen2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200C18CEED0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.01

**c.	Оценить качество имплементированных моделей по выбранным метрикам на выбранном наборе данных**

| Класс         | Precision | Recall | F1-score | mAP\@0.5 |
| ------------- | --------- | ------ | -------- | -------- |
| WBC      | 0.804     | 0.986  | 0.981    | 0.767    |
| RBC       | 0.624     | 0.914  | 0.817    | 0.541    |
| Platelets | 0.868     | 0.440  | 0.738    | 0.342    |


WBC (лейкоциты) демонстрируют лучшую полноту (recall = 0.986) и высокое значение F1 (0.981), что говорит о стабильной способности модели находить почти все объекты этого класса.
RBC (эритроциты) имеют более низкую точность (0.624), но высокую полноту (0.914), что означает большое количество ложноположительных срабатываний.
Platelets (тромбоциты) имеют самую высокую точность (0.868), но очень низкую полноту (0.440), что говорит о том, что модель видит лишь малую часть объектов этого класса.
mAP\@0.5 самый высокий у WBC (0.767), что делает его наиболее качественно предсказуемым классом. Самый низкий — у Platelets (0.342), что указывает на проблемы с локализацией или распознаванием.

**d.	Сравнить результаты имплементированных моделей в сравнении с результатами из пункта 2**

1. WBC (белые кровяные тельца)

| Модель        | Precision | Recall | F1-score | mAP\@0.5 |
| ------------- | --------- | ------ | -------- | -------- |
| Базовая       | 0.968     | 1.000  | 0.984    | 0.994    |
| Улучшенная    | 0.979     | 1.000  | 0.994    | 0.802    |
| Имплементация | 0.804     | 0.986  | 0.981    | 0.767    |

Улучшенная версия немного повысила точность по сравнению с базовой. Имплементация отстаёт по Precision и mAP\@0.5.


2. RBC (красные кровяные тельца)

| Модель        | Precision | Recall | F1-score | mAP\@0.5 |
| ------------- | --------- | ------ | -------- | -------- |
| Базовая       | 0.727     | 0.845  | 0.781    | 0.853    |
| Улучшенная    | 0.673     | 0.865  | 0.791    | 0.513    |
| Имплементация | 0.624     | 0.914  | 0.817    | 0.541    |

Имплементация лучше по Recall и F1-score, но уступает базовой в точности и mAP.

3. Platelets (тромбоциты)

| Модель        | Precision | Recall | F1-score | mAP\@0.5 |
| ------------- | --------- | ------ | -------- | -------- |
| Базовая       | 0.874     | 0.834  | 0.854    | 0.895    |
| Улучшенная    | 1.000     | 0.000  | 0.366    | 0.200    |
| Имплементация | 0.868     | 0.440  | 0.738    | 0.342    |

Улучшенная модель полностью провалилась по Recall и F1 для тромбоцитов. Имплементация уступает базовой, но всё же работает.

**e.	Сделать выводы**


Базовая модель — наиболее сбалансированная по всем классам.

Улучшенная модель — улучшила F1 и mAP для WBC, но серьёзно ухудшила распознавание тромбоцитов.

Имплементация — даёт высокий Recall по RBC и WBC, но страдает в точности и особенно по Platelets.

**f.	Добавить техники из улучшенного бейзлайна (пункт 3с)**


**g.	Обучить модели для выбранных наборов данных**

In [8]:
model_custom_aug = YOLO('yolov8n.pt')
model_custom_aug.train(data='dataset.yaml', epochs=5, imgsz=768, augment=True, project='lab8', name='custom_aug')

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True,

train: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\train.cache... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 131.626.1 MB/s, size: 20.9 KB)



val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]

Plotting labels to lab8\custom_aug\labels.jpg... 


...

                   all         73       1038      0.758      0.852      0.885      0.592



5 epochs completed in 0.355 hours.
Optimizer stripped from lab8\custom_aug\weights\last.pt, 6.2MB
Optimizer stripped from lab8\custom_aug\weights\best.pt, 6.2MB

Validating lab8\custom_aug\weights\best.pt...
Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:34<00:00, 11.51s/it]


                   all         73       1038      0.757      0.856      0.891      0.592
                   WBC         72         74      0.865          1      0.994       0.81
                   RBC         70        864      0.545      0.948      0.839      0.568
             Platelets         49        100      0.861      0.619      0.841      0.399
Speed: 2.2ms preprocess, 402.1ms inference, 0.0ms loss, 53.3ms postprocess per image
Results saved to lab8\custom_aug


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200BEFA9C10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

**h.	Оценить качество моделей по выбранным метрикам на выбранном наборе данных**

In [9]:
metrics_custom_aug = model_custom_aug.val()
print(metrics_custom_aug)

Ultralytics 8.3.130  Python-3.12.3 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 141.214.7 MB/s, size: 20.4 KB)


val: Scanning C:\Users\missm\AI-DA\sem2\dataset_yolo\labels\val.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:32<00:00,  6.51s/it]


                   all         73       1038      0.757      0.856      0.891      0.592
                   WBC         72         74      0.865          1      0.994       0.81
                   RBC         70        864      0.545      0.948      0.839      0.568
             Platelets         49        100      0.861      0.619      0.841      0.399
Speed: 1.5ms preprocess, 374.1ms inference, 0.0ms loss, 52.8ms postprocess per image
Results saved to lab8\custom_aug2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000200C1885550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.01201



| Класс               | Precision | Recall | mAP50 | F1    |
| ------------------- | --------- | ------ | ----- | ----- |
| **WBC**             | 0.865     | 1.000  | 0.994 | 0.927 |
| **RBC**             | 0.545     | 0.948  | 0.839 | 0.693 |
| **Platelets**       | 0.861     | 0.619  | 0.841 | 0.720 |





WBC (лейкоциты) определяются почти идеально.
RBC (эритроциты) хорошо детектируются (recall = 0.95), но модель часто ошибается (precision = 0.545), возможно, из-за сложности или схожести с фоном.
Platelets (тромбоциты) довольно точны при детекции (precision = 0.86), но модель пропускает много из них (recall = 0.619).

**i.	Сравнить результаты моделей в сравнении с результатами из пункта 3**

1. WBC (лейкоциты):
Все модели показывают очень высокую точность и полноту. Улучшенная модель достигла максимального F1-score (0.994), при этом базовая почти не отстаёт. Имплементация показывает чуть меньшую точность (Precision 0.804), но по F1-score (0.981) почти не уступает. Это говорит о высокой устойчивости моделей к распознаванию лейкоцитов.

2. RBC (эритроциты):
Тут различия более выражены. Имплементация имеет наивысший F1-score (0.817) и Recall (0.914), что говорит о способности находить почти все объекты класса. Однако Precision у неё ниже (0.624), что указывает на частые ложные срабатывания. Базовая модель демонстрирует лучший баланс между Precision и mAP@0.5, что может быть важно для задач с высокими требованиями к точности.

3. Platelets (тромбоциты):
Результаты сильно варьируются между моделями. Улучшенная модель показала полный провал по Recall (0.000), следовательно, она вообще не распознала ни одного тромбоцита. Базовая модель показывает наилучший результат с F1-score 0.854. Имплементация же справляется умеренно — точность и F1-score заметно ниже, чем у базовой, но лучше, чем у улучшенной.

**j.	Сделать выводы**

Базовая модель продемонстрировала наиболее стабильные и высокие результаты во всех трёх классах, особенно в задачах, где важен баланс между Precision и Recall. Улучшенная модель оказалась эффективной только для WBC, но показала неудовлетворительные результаты для Platelets. Имплементация показала неплохой Recall и F1-score для RBC, однако в целом уступает базовой по большинству метрик. Следовательно, базовая модель является наилучшим выбором среди рассмотренных, особенно для задач, требующих универсальности и надёжности.